In [2]:
%load_ext autoreload
%autoreload 2

In [43]:
import polars as pl
import pandas as pd
import seaborn as sns

In [39]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

print(train.shape, test.shape)
display(train.head(2))
display(test.head(2))


(891, 12) (418, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1198,0,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0,1,2,113781,151.550,C22 C26,S
1,1094,0,1,"Astor, Col. John Jacob",male,47.0,1,0,PC 17757,227.525,C62 C64,C


In [40]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [41]:
train.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [42]:
high_ordinality_cols = ['PassengerId', 'Name', 'Age', 'Ticket', 'Fare', 'Cabin', 'Survived']
for feat1_index in range(len(train.columns)):
    for feat2_index in range(feat1_index+1, len(train.columns)):
        if (train.columns[feat1_index] not in high_ordinality_cols) & (train.columns[feat2_index] not in high_ordinality_cols):
            print('='*50)
            print(train.columns[feat1_index], train.columns[feat2_index])

            print('Train DF:')
            display(
                train[[train.columns[feat1_index], train.columns[feat2_index], 'Survived']]
                .groupby([train.columns[feat1_index], train.columns[feat2_index]], as_index=False)
                .agg(
                    SurvivalRate = ('Survived', 'mean'), 
                    GroupCount = ('Survived', 'count'), 
                )
            )
            print('Test DF:')
            display(
                test[[test.columns[feat1_index], test.columns[feat2_index], 'Survived']]
                .groupby([test.columns[feat1_index], test.columns[feat2_index]], as_index=False)
                .agg(
                    SurvivalRate = ('Survived', 'mean'), 
                    GroupCount = ('Survived', 'count'), 
                )
            )

Pclass Sex
Train DF:


,Pclass,Sex,SurvivalRate,GroupCount
0,1,female,0.968085,94
1,1,male,0.368852,122
2,2,female,0.921053,76
3,2,male,0.157407,108
4,3,female,0.500000,144
5,3,male,0.135447,347


Test DF:


,Pclass,Sex,SurvivalRate,GroupCount
0,1,female,0.960000,50
1,1,male,0.280702,57
2,2,female,0.800000,30
3,2,male,0.126984,63
4,3,female,0.472222,72
5,3,male,0.191781,146


Pclass SibSp
Train DF:


...

,SibSp,Embarked,SurvivalRate,GroupCount
0,0,C,0.548387,62
1,0,Q,0.292683,41
2,0,S,0.294444,180
3,1,C,0.594595,37
4,1,Q,0.333333,3
5,1,S,0.400000,70
6,2,C,0.333333,3
7,2,Q,1.000000,1
8,2,S,0.400000,10
9,3,S,0.500000,4


Parch Embarked
Train DF:


,Parch,Embarked,SurvivalRate,GroupCount
0,0,C,0.512195,123
1,0,Q,0.434783,69
2,0,S,0.285124,484
3,1,C,0.633333,30
4,1,Q,0.000000,6
5,1,S,0.560976,82
6,2,C,0.714286,14
7,2,Q,0.000000,1
8,2,S,0.461538,65
9,3,C,1.000000,1


Test DF:


,Parch,Embarked,SurvivalRate,GroupCount
0,0,C,0.452055,73
1,0,Q,0.311111,45
2,0,S,0.271845,206
3,1,C,0.857143,21
4,1,Q,0.000000,1
5,1,S,0.566667,30
6,2,C,0.833333,6
7,2,S,0.444444,27
8,3,C,0.500000,2
9,3,S,1.000000,1


In [50]:
# train.head()

In [52]:
#EDA for non numeric features
# sns.histplot(data=train, x="Age", hue='Sex')
# sns.histplot(data=train, x="Fare", hue='Sex')